In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge

In [ ]:

df = pd.read_csv("/kaggle/input/refined-dataset2/best_features.csv")
label = pd.read_csv("/kaggle/input/refined-dataset2/data.csv")
label

In [ ]:
sc = ['winsorized_Adult_Mortality',
 'winsorized_Schooling',
 'winsorized_Diphtheria',
 'winsorized_Percentage_Exp',
 'winsorized_Infant_Deaths',
 'winsorized_GDP',
 'winsorized_Income_Comp_Of_Resources',
 'winsorized_Tot_Exp',
 'winsorized_thinness_5to9_years',
 'winsorized_thinness_1to19_years']

In [ ]:
# example of a normalization
from numpy import asarray
from sklearn.preprocessing import MinMaxScaler
from numpy import asarray
from sklearn.preprocessing import StandardScaler
# define data
for i in sc:
# define standard scaler

    scaler = MinMaxScaler()

    df[[i]] = scaler.fit_transform(df[[i]])


In [ ]:
numerical_columns = [x for x in list(df.columns) if x.startswith("winsorized")]
numerical_columns

In [ ]:
max(df["winsorized_Adult_Mortality"])

In [ ]:
numerical_columns = list(set(numerical_columns) - set("winsorized_Life_Expectancy"))
y = "winsorized_Life_Expectancy"
X_train = np.array(df[numerical_columns])
y_train = np.array(label[y])
prediction = np.array(df[numerical_columns].iloc[0].values)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train,y_train ,
                                   random_state=1100, 
                                   test_size=0.25, 
                                   shuffle=True)

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
tscv = TimeSeriesSplit(n_splits=5)
i = 1
score = []
for tr_index, val_index in tscv.split(X_train):
    X_tr, X_val = X_train[tr_index], X_train[val_index]
    y_tr, y_val = y_train[tr_index], y_train[val_index]
    for al in np.linspace(10, 1000, 6):
        for mi in np.linspace(50, 100, 6):
            for tol in np.linspace(20, 40, 5):
                rfr = Ridge(
                    alpha=int(al),
                    max_iter=int(mi),
                    tol=tol)
                rfr.fit(X_tr, y_tr)
                score.append([i,
                              al, 
                              mi,
                              tol, 
                              rfr.score(X_val, y_val) ,  r2_score(rfr.predict(X_val ) , y_val)])
    i += 1

In [ ]:
score[0]
max(score , key = lambda x : x[4])

In [ ]:
model = Ridge(alpha=10.0,max_iter=int(50.0),tol=20.0)

In [ ]:
max_score = 0
for tr_index, val_index in tscv.split(X_train):
    X_tr, X_val = X_train[tr_index], X_train[val_index]
    y_tr, y_val = y_train[tr_index], y_train[val_index]
    rfr = model
    rfr.fit(X_tr, y_tr)
    if rfr.score(X_val, y_val) > max_score:
        model = rfr
                
                

In [ ]:
rfr.predict([prediction])

In [ ]:
import pickle
pickle.dump(rfr, open('model_baseline_Ridge.pkl','wb'))